<a href="https://colab.research.google.com/github/SchniedersLab/forcefieldx/blob/master/colab/ForceFieldX.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GPU-Accelerated Protein Repacking in Force Field X 

### 1) Check the Java and CUDA versions.

In [0]:
# Import os to set environment variables.
import os

# Check the version of Java: JDK 10 or higher required.
!java -version

# Uncomment the next two line to install openjdk version 11 if necessary.
# !apt-get install -y openjdk-11-jdk-headless -qq > /dev/null
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"

# Check the GPU/CUDA combination -- tested with Tesla P100, Driver 418.67 and CUDA 10.1.
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!nvidia-smi

### 2) Download Force Field X from the U. of Iowa.

In [0]:
!rm -rf ffx-1.0.0-beta
!wget https://ffx.biochem.uiowa.edu/ffx-1.0.0-beta-bin.tar.gz
!tar xzf ffx-1.0.0-beta-bin.tar.gz
!rm ffx-1.0.0-beta-bin.tar.gz
os.environ["PATH"] = "ffx-1.0.0-beta/bin:" + os.environ["PATH"]

### 3) GPU-Accelerated Side-Chain Optimization Using the AMOEBA force field.

In [0]:
# Please cite:
# Tollefson, M. R.;  Litman, J. M.;  Qi, G.;  O’Connell, C. E.;  Wipfler, M. J.;  Marini, R. J.;  
# Bernabe, H. V.;  Tollefson, W. T. A.;  Braun, T. A.;  Casavant, T. L.;  Smith, R. J. H.; Schnieders, M. J., 
# Structural insights into hearing loss genetics from polarizable protein repacking. Biophys. J. 2019, 117 (3), 602-612.
#
# Choose a PDB file from the Protein Databank.
os.environ["PDB"] = "2JOF.pdb"
!wget https://files.rcsb.org/download/${PDB}

# Run the many-body side-chain optimization using the polarizable AMOEBA force field.
!ffxc ManyBody ${PDB} -Dplatform=omm

# Locally optimize the result.
!ffxc MinimizeOpenMM -e 0.1 ${PDB}_2

### 4) Download the result and clean up.

In [0]:
from google.colab import files
files.download(os.environ["PDB"] + "_3")
!rm ${PDB}* *.restart
!rm -rf ffx-1.0.0-beta